Work In Progress

In this post, we demonstrate how to use Gaussian Mixture Models to cluster the factors extracted from a rolling PCA over daily equity return data.

## Setup


In [ ]:
#| echo: false
#| code-fold: true
#| code-summary: "Auto reload"
%load_ext autoreload
%autoreload 2

In [ ]:
#| echo: false
#| code-fold: true
#| code-summary: "Environment"
import os
import sys
import importlib
sys.path.append("C:/hc/src")
sys.path.append("C:/hc/rambling")
sys.path.append("C:/hc/xfactors/src")
sys.path.append("C:/hc/xtuples/src")
os.environ["MODULE"] = "c:/hc/src/"

In [ ]:
#| code-fold: false
#| code-summary: "Imports"
import numpy
import pandas
import jax
import jax.numpy

import xtuples as xt
import xfactors as xf

import hcbt.data.prices.int as prices
import hcbt.algos.universe.int as universe
import hcbt.algos.universe.configs as configs

### Data


In [ ]:
#| code-fold: false
#| code-summary: "Returns"
def vol_scale(df, alpha = 2 / 90):
    std = df.ewm(alpha = alpha).std()
    return df.divide(std)

In [ ]:
#| code-fold: false
#| code-summary: "Returns"
df_returns = prices.returns_df(
    xf.utils.dates.y(2005),
    xf.utils.dates.y(2023, m=4),
    indices=configs.INDICES,
)
# df_returns = vol_scale(vol_scale)

In [ ]:
#| code-fold: false
#| code-summary: "Returns"
df_returns.head()

The returns are from bloomberg, and include all cash and non-cash adjustments (in production we have a separate internal ticker for the dividend stream from a given name, but that's a little over-complicated for our purposes here).

We'll also load the relevant index membership mapping tables for our universe:


In [ ]:
#| code-fold: false
#| code-summary: "Index Membership"
dfs_indices = universe.rolling_indices(
    xf.utils.dates.y(2005),
    xf.utils.dates.y(2023, m=4),
    indices=configs.INDICES,
)
df_universe = universe.index_union(dfs_indices)

In [ ]:
#| code-fold: false
#| code-summary: "Sector Membership"
dfs_sectors = universe.rolling_indices(
    xf.utils.dates.y(2005),
    xf.utils.dates.y(2023, m=4),
    sectors=configs.GICS_SECTORS,
)

### Model


Rolling pca -> clustering


In [ ]:
#| code-fold: false
#| code-summary: "Model"
N_STAGES = 5
model, STAGES = xf.Model().init_stages(N_STAGES)
INPUTS, SETUP, PCA, FLATTEN, PARAMS, GMM = STAGES

def define_model(n_factors):
    model, stages = xf.Model().init_stages(N_STAGES)
    assert stages.len() == STAGES.len()

    n = n_factors

    model = (
        model.add_input(xf.nodes.inputs.dfs.Input_DataFrame_Wide_Rolling(
            step=2,
            window=4,
            unit="M",
            allow_missing_columns=False,
            allow_missing_indices=True,
            allow_new_columns=False,
            allow_new_indices=True,
            na_threshold_columns=0.,
            na_threshold_indices=0.,
        ))
        .add_input(xf.nodes.inputs.dfs.Input_DataFrame_Wide(
            allow_missing_columns=True,
            allow_missing_indices=False,
            allow_new_columns=True,
            allow_new_indices=False,
            na_threshold_columns=0.,
            na_threshold_indices=0.,
        ))
        .add_node(SETUP, xf.nodes.inputs.dfs.Slice_DataFrame_Wide_Rolling_Columns(
            rolling=xf.Loc.result(INPUTS, 0),
            slicing=xf.Loc.result(INPUTS, 1),
            scale=xf.utils.scaling.Unit_Sum(axis=1)
        ), static = True)
        .add_node(SETUP, xf.nodes.cov.vanilla.VCov(
            data=xf.Loc.result(INPUTS, 0),
        ), static = True)
        .add_node(PCA, xf.nodes.pca.rolling.PCA_Rolling(
            n=n,
            data=xf.Loc.result(INPUTS, 0),
        ), static=True)

        # TODO: STRUCTURE - need to map up into the same size space
        # ie. sector average

        .add_node(FLATTEN, xf.nodes.control.shapes.Concatenate(
            axis=0,
            loc=xf.Loc.result(PCA, 0),
        ), static=True)
        .add_node(PARAMS, xf.nodes.params.random.Gaussian(
            shape=(N_CLUSTERS, N_COLS,),
        ))
        .add_node(PARAMS, xf.nodes.params.random.Gaussian(
            shape=(N_CLUSTERS, N_COLS, N_COLS,),
        ))
        # .add_node(PARAMS, xf.nodes.params.random.GaussianSoftmax(
        #     shape=(data[0].shape[0], N_CLUSTERS,),
        # ))
        .add_node(PARAMS, xf.nodes.params.random.GaussianSoftmax(
            shape=(N_CLUSTERS,),
        ))
        .add_node(GMM, xf.nodes.clustering.gmm.BGMM_EM(
            k=N_CLUSTERS,
            data=xf.Loc.result(FLATTEN, 0),
            mu=xf.Loc.param(PARAMS, 0),
            cov=xf.Loc.param(PARAMS, 1),
        ), random = True)
        .add_constraint(xf.nodes.constraints.loss.Constraint_Maximise(
            data=xf.Loc.result(GMM, 0, 1),
        ))
        .add_constraint(xf.nodes.constraints.loss.Constraint_Maximise(
            data=xf.Loc.result(GMM, 0, 2),
        ))
        .add_constraint(xf.nodes.constraints.linalg.Constraint_VOrthogonal(
            data=xf.Loc.param(PARAMS, 1),
        ))
        .add_constraint(xf.nodes.constraints.linalg.Constraint_L1_MM_Diag(
            raw=xf.Loc.param(PARAMS, 1),
        ))

    return model

In [ ]:
#| code-fold: false
#| code-summary: "Run"
def fit_model(d_start, d_end, n_factors):

    (lambda _df: _df.assign(
        index=xf.utils.dates.date_index(_df.index.values)
    ).set_index("index"))(df_returns.loc[
        (df_returns.index >= d_start) & (df_returns.index <= d_end)
    ]),

    model = define_model(n_factors).init(data).optimise(
        data,
        iters = 2500,
        rand_init=10, 
        max_error_unchanged=0.5,
    )
    results = model.apply(data)
    params = model.params[PARAMS]
    
    mu = params[0]
    cov = params[1]
    probs = results[GMM][0][0]
    
    cov = numpy.round(numpy.matmul(
        numpy.transpose(cov, (0, 2, 1)),
        cov,
    ), 3)

    labels = probs.argmax(axis=1)

    # group structured average (ie. sector average of pc weights)
    # by cluster label

    # and then plot against cluster mu

    # separately, per cluster, plot the covar

    return df, 